In [1]:
import pandas as pd
import numpy as np
import ast

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder

In [2]:
geo_info = pd.read_csv('geo_info.csv', sep=';')
referer_vectors = pd.read_csv('referer_vectors.csv', sep=';')
test_users = pd.read_csv('test_users.csv', sep=';')
test = pd.read_csv('test.csv', sep=';')
train_labels = pd.read_csv('train_labels.csv', sep=';')
train = pd.read_csv('train.csv', sep=';')

referer_vectors = referer_vectors.drop_duplicates()

In [3]:
# Функция для безопасного извлечения данных из user_agent
def extract_user_agent_info(user_agent, key):
    try:
        if pd.notna(user_agent):
            return ast.literal_eval(user_agent).get(key, np.nan)
        else:
            return np.nan
    except (ValueError, SyntaxError):
        return np.nan

# Обработка данных train
train['request_ts'] = pd.to_datetime(train['request_ts'], unit='s')
train['hour'] = train['request_ts'].dt.hour
train['day_of_week'] = train['request_ts'].dt.dayofweek
train['browser'] = train['user_agent'].apply(lambda x: extract_user_agent_info(x, 'browser'))
train['os'] = train['user_agent'].apply(lambda x: extract_user_agent_info(x, 'os'))
train['browser_version'] = train['user_agent'].apply(lambda x: extract_user_agent_info(x, 'browser_version'))
train['os_version'] = train['user_agent'].apply(lambda x: extract_user_agent_info(x, 'os_version'))

# Обработка данных test
test['request_ts'] = pd.to_datetime(test['request_ts'], unit='s')
test['hour'] = test['request_ts'].dt.hour
test['day_of_week'] = test['request_ts'].dt.dayofweek
test['browser'] = test['user_agent'].apply(lambda x: extract_user_agent_info(x, 'browser'))
test['os'] = test['user_agent'].apply(lambda x: extract_user_agent_info(x, 'os'))
test['browser_version'] = test['user_agent'].apply(lambda x: extract_user_agent_info(x, 'browser_version'))
test['os_version'] = test['user_agent'].apply(lambda x: extract_user_agent_info(x, 'os_version'))

In [4]:
old_shapes = [test.shape, train.shape]

# Присоединение информации о referer и geo
train = train.merge(referer_vectors, on='referer', how='left')
train = train.merge(geo_info, on='geo_id', how='left')

test = test.merge(referer_vectors, on='referer', how='left')
test = test.merge(geo_info, on='geo_id', how='left')

# Проверяем результат преобразований
print(f'Old: {old_shapes}, New: {test.shape, train.shape}')


Old: [(150000, 11), (750000, 11)], New: ((150000, 24), (750000, 24))


In [5]:
train_df = train.merge(train_labels, on='user_id', how='left')
train_df['target'].value_counts(dropna=False)

0.0    309409
1.0    284034
NaN    156557
Name: target, dtype: int64

In [6]:
# Выбираем все доступные признаки для обучения
features = [
    'hour', 'day_of_week', 'browser', 'os', 'os_version',
    'component0', 'component1', 'component2', 'component3', 'component4',
    'component5', 'component6', 'component7', 'component8', 'component9',
    'country_id', 'region_id', 'timezone_id'
]
X = train_df[features]
y = train_df['target']

X = X[~y.isna()]
y = y[~y.isna()]

In [7]:

# Кодируем целевую переменную, если она категориальная
y_encoded = LabelEncoder().fit_transform(y)

# Убедимся, что категориальные признаки закодированы в числовой формат
X_categorical = X[['country_id', 'region_id']].apply(LabelEncoder().fit_transform)

# Применение хи-квадрат теста
chi2_stats, p_values = chi2(X_categorical, y_encoded)

# Создание DataFrame для визуализации значимости
chi2_results = pd.DataFrame({
    'feature': ['country_id', 'region_id'],
    'chi2_stat': chi2_stats,
    'p_value': p_values
})

# Вывод результатов
print(chi2_results.sort_values(by='p_value'))

      feature    chi2_stat  p_value
0  country_id  2010.230195      0.0
1   region_id  2921.141304      0.0


In [31]:
# Кодируем категориальные признаки
X = pd.get_dummies(X, columns=['browser', 'os', 'os_version',
                               'country_id', 'region_id', 'timezone_id'], drop_first=True)

# Разделяем на обучающую и тестовую выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Создание и обучение модели случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Предсказание и оценка модели
y_pred = model.predict(X_valid_scaled)
print(f"Accuracy: {accuracy_score(y_valid, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_valid, y_pred))

Accuracy: 0.8233
Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.84      0.83     61922
         1.0       0.82      0.81      0.81     56767

    accuracy                           0.82    118689
   macro avg       0.82      0.82      0.82    118689
weighted avg       0.82      0.82      0.82    118689



In [33]:
# Получение важности признаков
feature_importances = model.feature_importances_

# Создание DataFrame для визуализации важности признаков
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': feature_importances
})

# Сортировка признаков по важности
importance_df = importance_df.sort_values(by='importance', ascending=False)

# Вывод значимости первых 10 признаков
print(importance_df.head(50))

                           feature  importance
0                             hour    0.113313
3                       component1    0.083520
10                      component8    0.069875
4                       component2    0.068117
5                       component3    0.064612
7                       component5    0.057350
2                       component0    0.056745
8                       component6    0.056678
11                      component9    0.056415
9                       component7    0.055370
6                       component4    0.051184
69            browser_YandexSearch    0.010975
67          browser_Yandex Browser    0.010010
79                      os_Windows    0.009218
17                  browser_Chrome    0.008544
699               region_id_8ccc01    0.007262
82                   os_version_10    0.007059
18           browser_Chrome Mobile    0.007059
1                      day_of_week    0.006317
19   browser_Chrome Mobile WebView    0.005833
712          

In [8]:
# Выбираем все доступные признаки для обучения
features = [
    'hour', 'day_of_week', 'browser', 'os',
    'component0', 'component1', 'component2', 'component3', 'component4',
    'component5', 'component6', 'component7', 'component8', 'component9',
    'country_id', 'region_id', 'timezone_id'
]
X = train_df[features]
y = train_df['target']

X = X[~y.isna()]
y = y[~y.isna()]

# Кодируем категориальные признаки
X = pd.get_dummies(X, columns=['browser', 'os',
                               'country_id', 'region_id', 'timezone_id'], drop_first=True)

# Разделяем на обучающую и тестовую выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Создание и обучение модели случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Предсказание и оценка модели
y_pred = model.predict(X_valid_scaled)
print(f"Accuracy: {accuracy_score(y_valid, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_valid, y_pred))

Accuracy: 0.8245
Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.84      0.83     61922
         1.0       0.82      0.81      0.82     56767

    accuracy                           0.82    118689
   macro avg       0.82      0.82      0.82    118689
weighted avg       0.82      0.82      0.82    118689



In [19]:
# Выбираем все доступные признаки для обучения
features = [
    'hour', 'day_of_week', 'browser', 'os',
    'component0', 'component1', 'component2', 'component3', 'component4',
    'component5', 'component6', 'component7', 'component8', 'component9',
    'country_id', 'region_id', 'timezone_id'
]
X = train_df[features]
y = train_df['target']

X = X[~y.isna()]
y = y[~y.isna()]

# Убедимся, что категориальные признаки закодированы в числовой формат
X[['browser', 'os', 'country_id', 'region_id', 'timezone_id']] = X[['browser', 'os', 'country_id', 'region_id', 'timezone_id']].apply(LabelEncoder().fit_transform)

# Разделяем на обучающую и тестовую выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Создание и обучение модели случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Предсказание и оценка модели
y_pred = model.predict(X_valid_scaled)
print(f"Accuracy: {accuracy_score(y_valid, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_valid, y_pred))

Accuracy: 0.8244
Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.84      0.83     61922
         1.0       0.82      0.81      0.81     56767

    accuracy                           0.82    118689
   macro avg       0.82      0.82      0.82    118689
weighted avg       0.82      0.82      0.82    118689



In [20]:
# Выбираем все доступные признаки для обучения
features = [
    'hour', 'day_of_week', 'browser', 'os', 'os_version',
    'component0', 'component1', 'component2', 'component3', 'component4',
    'component5', 'component6', 'component7', 'component8', 'component9',
    'country_id', 'region_id', 'timezone_id'
]
X = train_df[features]
y = train_df['target']

X = X[~y.isna()]
y = y[~y.isna()]

# Убедимся, что категориальные признаки закодированы в числовой формат
X[['browser', 'os', 'os_version', 'country_id', 'region_id', 'timezone_id']] = X[['browser', 'os', 'os_version', 'country_id', 'region_id', 'timezone_id']].apply(LabelEncoder().fit_transform)

# Разделяем на обучающую и тестовую выборки
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Создание и обучение модели случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Предсказание и оценка модели
y_pred = model.predict(X_valid_scaled)
print(f"Accuracy: {accuracy_score(y_valid, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_valid, y_pred))

Accuracy: 0.8252
Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.84      0.83     61922
         1.0       0.82      0.81      0.82     56767

    accuracy                           0.83    118689
   macro avg       0.83      0.82      0.82    118689
weighted avg       0.83      0.83      0.83    118689



# Краткий вывод:

1. Случайный лес со всеми признаками, кодирование категориальных с помощью лейблов

**Accuracy: 0.8252**

2. Случайный лес без признака os_version, кодирование категориальных с помощью лейблов

**Accuracy: 0.8244**

3. Случайный лес со всеми признаками, кодирование категориальных с помощью One_hot

**Accuracy: 0.8233**

4. Случайный лес без признака os_version, кодирование категориальных с помощью One_hot

**Accuracy: 0.8245**